In [2]:
import pandas as pd
import time

df_train = pd.read_csv('../data/processed/loan_e_p_cleaned_train.csv')
df_test = pd.read_csv('../data/processed/loan_e_p_cleaned_test.csv')

In [3]:
df_train.shape

(491, 31)

In [4]:
df_test.shape

(123, 31)

In [11]:
def format_duration(seconds):
    if seconds < 60:
        return f'{seconds:.2f}s'
    elif seconds < 3600:
        minutes = int(seconds // 60)
        sec = seconds % 60
        return f'{minutes}m : {sec:.2f}s'
    else:
        hours = int(seconds // 3600)
        minutes = int((seconds % 3600) // 60)
        secs = int(seconds % 60)
        return f'{hours}h : {minutes}m : {secs}s'

In [14]:
start = time.time()
y_train = df_train['Loan_Status']
print(y_train.shape)
x_train = df_train.drop(columns='Loan_Status').copy()
print(x_train.shape)
time_elapsed = time.time() - start
print(format_duration(time_elapsed))

(491,)
(491, 30)
0.01s


In [40]:
from sklearn.inspection import permutation_importance
import shap     
from sklearn.feature_selection import VarianceThreshold

start = time.time()
selector = VarianceThreshold(threshold=0.01)
selector.fit(x_train)
time_elapsed = time.time() - start
print(format_duration(time_elapsed))

0.01s


In [41]:
y_test = df_test['Loan_Status']
print(y_test.shape)
x_test = df_test.drop(columns='Loan_Status').copy()
print(x_test.shape)

(123,)
(123, 30)


In [44]:
selected_mask = selector.get_support()
selected_cols = x_train.columns[selected_mask].tolist()
x_train = x_train[selected_cols]
x_test = x_test[selected_cols]
print(x_train.shape)
print(x_test.shape)

(491, 18)
(123, 18)


In [45]:
import numpy as np
corr_matrix = x_train.corr(method='spearman')
upper_triangle = corr_matrix.where(
    np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
)

to_drop = [col for col in upper_triangle.columns if any(upper_triangle[col] > 0.95)]
x_train.drop(columns=to_drop, inplace=True)
x_test.drop(columns=to_drop, inplace=True)
print(x_train.shape)
print(x_test.shape)

(491, 18)
(123, 18)


In [ ]:
# train baseline models
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score
from xgboost import XGBClassifier
import lightgbm
from sklearn.ensemble import RandomForestClassifier